In [1]:
import findspark
findspark.init('/home/ubuntu/spark-2.1.1-bin-hadoop2.7')
import pyspark

/home/ubuntu/spark-2.1.1-bin-hadoop2.7/python/pyspark/cloudpickle.py:365: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if obj.__module__ is "__builtin__":


In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.appName('dogfood').getOrCreate()

In [4]:
data = spark.read.csv('dog_food.csv',inferSchema=True,header=True)

In [5]:
data.head(1)

[Row(A=4, B=2, C=12.0, D=3, Spoiled=1.0)]

In [6]:
from pyspark.ml.feature import VectorAssembler

In [7]:
data.columns

['A', 'B', 'C', 'D', 'Spoiled']

In [8]:
assembler = VectorAssembler(inputCols=['A', 'B', 'C', 'D'],
                           outputCol='features')

In [9]:
output = assembler.transform(data)

In [10]:
from pyspark.ml.classification import RandomForestClassifier

In [11]:
rfc = RandomForestClassifier(labelCol='Spoiled',featuresCol='features')

In [12]:
output.printSchema()

root
 |-- A: integer (nullable = true)
 |-- B: integer (nullable = true)
 |-- C: double (nullable = true)
 |-- D: integer (nullable = true)
 |-- Spoiled: double (nullable = true)
 |-- features: vector (nullable = true)



In [13]:
final_data = output.select('features','Spoiled')

In [14]:
rfc_model = rfc.fit(final_data)

In [15]:
rfc_model.featureImportances

SparseVector(4, {0: 0.024, 1: 0.021, 2: 0.9362, 3: 0.0188})

In [ ]:
# feature 2 = chemical C is causing the early spoilage 